In [16]:
import psycopg2
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)
import matplotlib
import warnings
import yellowbrick as yb

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, NuSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression, SGDClassifier
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier

from yellowbrick.classifier import ClassificationReport

warnings.simplefilter(action='ignore', category=FutureWarning)

In [17]:
conn = psycopg2.connect( 
                        host = 'project.cgxhdwn5zb5t.us-east-1.rds.amazonaws.com',
                        port = 5432, 
                        user = 'postgres',
                        password = 'Admin123',
                        database = 'postgres')
cursor = conn.cursor()

In [18]:
DEC2FLOAT = psycopg2.extensions.new_type(
    psycopg2.extensions.DECIMAL.values,
    'DEC2FLOAT',
    lambda value, curs: float(value) if value is not None else None)
psycopg2.extensions.register_type(DEC2FLOAT)

In [19]:
cursor.execute('Select * from "ahshouseholdclass2"')
rows = cursor.fetchall()
col_names = []
for elt in cursor.description:
    col_names.append(elt[0])

df = pd.DataFrame(data=rows, columns=col_names )

In [20]:
LABEL_MAP = {
"(90000.0, 100000.0]": 10,
"(80000.0, 90000.0]": 9,
"(70000.0, 80000.0]": 8,   
"(60000.0, 70000.0]": 7,
"(50000.0, 60000.0]": 6,
"(40000.0, 50000.0]": 5,
"(30000.0, 40000.0]": 4,
"(20000.0, 30000.0]": 3,
"(10000.0, 20000.0]": 2,
"(0.0, 10000.0]": 1}

# Convert categorical labels into incremental value
df['HINCP_BIN'] = df['HINCP_BIN'].map(LABEL_MAP).to_frame()

In [21]:
# Create the train and test data
from sklearn.model_selection import train_test_split

X = df[['HHAGE', 'HINCP', 'UTILAMT','UNITSIZE','NUMPEOPLE','BATHROOMS', 
        'PERPOVLVL',  'ELECAMT', 'GASAMT', 
        'TRASHAMT', 'WATERAMT', 'OMB13CBSA','STORIES', 'HHNATVTY'
       ]]

y = df['RATINGHS_BIN']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

In [25]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

import pickle

# save the model to disk
filename = 'test_model.sav'
